Bem vindo a Pokedex de batalha.

Aqui, com o intuito de montar um time competitivo de batalha, para qualquer geração da franquia,
apresento uma ferramenta de busca tanto de pokémons, quanto de movimentos e TMs.

Documentação do Código
Descrição
Este script coleta dados de três diferentes endpoints da API do Pokémon (PokéAPI), armazena esses dados em um 
banco de dados, realiza algumas manipulações básicas nos dados e o converte para o formato CSV.
O script é dividido em três partes principais:

Coleta de dados de Pokémons.
Coleta de dados de TMs.
Coleta de dados de Movimentos.

Além disso, o script possui funções auxiliares para salvar dados no banco de dados e enviar notificações.

Dependências:
Este script depende das seguintes bibliotecas:
pandas
requests
sqlite3
datetime

Coleta de Dados
1. Coleta de Dados dos Pokémons
Coleta dados dos Pokémons da PokéAPI e salva no banco de dados.
Dados de cada espécie de pokémon, são retirados da API "https://pokeapi.co/api/v2/pokemon", e categorizados em:
Name - Nome da espécie
Height - Altura do pokémon 
Weight - Peso do pokémon (importante para moves como Low Kick e Grass Knot)
Type - O tipo de cada pokémon, podendo ter até dois
Base Experience - Parâmetro que influencia na taxa de crescimento do pokémon

2. Coleta de Dados de TMs
Coleta dados de TMs da PokéAPI e salva no banco de dados.
Dados de cada TM de versões diferentes do jogo, retirado da API "https://pokeapi.co/api/v2/machine", e categorizados em:
TM: Número da TM
Generation: Geração da franquia ao qual esta TM está listada
Move: Nome do movimento

3. Coleta de Dados de Movimentos
Coleta dados de Movimentos da PokéAPI e salva no banco de dados.
Dados de cada movimentos de pokémons, retirados da API "https://pokeapi.co/api/v2/move" e categorizados em:
Nome: Nome
Type: Tipo
Power: Poder base
Accuracy: Precisão
Power Points: PP (número de usos)
Effect: Pequena descrição do movimento e seus efeitos secundários.

Consulta das Tabelas no Banco de Dados
Consulta e exibe os nomes das tabelas presentes no banco de dados.

Conclusão
Este script fornece uma maneira automatizada de coletar, processar e armazenar dados da PokéAPI em um
banco de dados CSV, tornando possível seu consumo no SQL Server para uma análise aprofundada de dados.

Importando Bibliotecas

In [ ]:
import pandas as pd
import requests
import sqlite3

Função para salvar o banco de dados

In [ ]:
def save_to_db(df, table_name, conn):
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Tabela {table_name} salva no banco de dados")

# Pokemons

Consumo de APIs e criação do Data Frame

In [ ]:
url = "https://pokeapi.co/api/v2/pokemon"
response = requests.get(url)

In [ ]:
#verificação da resposta da funcionalidade da API
if response.status_code == 200:
    data_json = response.json()
    print("Funcionou")

    pokemon_list = []
    for pokemon in data_json['results']:
        pokemon_url = pokemon['url']
        pokemon_response = requests.get(pokemon_url)
        if pokemon_response.status_code == 200:
            pokemon_data = pokemon_response.json()            
        
            types = ", ".join([type_info['type']['name'] for type_info in pokemon_data['types']])
            
            pokemon_info = {
                'name': pokemon_data['name'],
                'height': pokemon_data['height'],
                'weight': pokemon_data['weight'],
                'types': types,
                'base_experience': pokemon_data['base_experience']
            }
            pokemon_list.append(pokemon_info)
        else:
            print(f"Falha ao acessar dados de {pokemon['name']}")

    df_pokemons = pd.DataFrame(pokemon_list)
else:
    print("Não foi")

Limpeza de dados

In [ ]:
df_pokemons.duplicated()
df_pokemons.drop_duplicates()

In [ ]:
df_pokemons.groupby('height')
df_pokemons.sort_values('height', ascending=True)

Criação de Banco de Dados e conexão com SQL

In [ ]:
conn = sqlite3.connect('pokedex.db')
save_to_db(df_pokemons, 'Pokemon', conn)
conn.close()

In [ ]:
conn = sqlite3.connect('pokedex.db')
query = "SELECT * FROM Pokemon"
df = pd.read_sql(query, conn)
print(df)

conn.close()

# TMs

Consumo de APIs e criação do Data Frame

In [ ]:
url = "https://pokeapi.co/api/v2/machine"
response = requests.get(url)

In [ ]:
if response.status_code == 200:
    data_json = response.json()
    print("Funcionou")

    machine_list = []
    for machine in data_json['results']:
        machine_url = machine['url']
        machine_response = requests.get(machine_url)
        machine_data = machine_response.json()
        machine_info = {
             'id': machine_data['id'],
             'TM': machine_data['item']['name'],
             'generation': machine_data['version_group']['name'],
             'move': machine_data['move']['name'],

            }
        machine_list.append(machine_info)
    df_machine_list = pd.DataFrame(machine_list)
    
else:
    print("Não foi")

Limpeza de dados

In [ ]:
df_machine_list.duplicated()
df_machine_list.drop_duplicates()

Criação de Banco de Dados e conexão com SQL

In [ ]:
conn = sqlite3.connect('pokedex.db')
save_to_db(df_machine_list, 'TMs', conn)
conn.close()

In [ ]:
conn = sqlite3.connect('pokedex.db')
query = "SELECT * FROM TMs"
df = pd.read_sql(query, conn)
print(df)

conn.close()

# Moves

Consumo de APIs e criação do Data Frame

In [ ]:
url = "https://pokeapi.co/api/v2/move"
response = requests.get(url)

In [ ]:
if response.status_code == 200:
    data_json = response.json()
    print("funcionou")

    move_list = []
    for move in data_json ['results']:
        move_url = move['url']
        move_response = requests.get(move_url)
        move_data = move_response.json()
        move_info = {
            'name': move_data ['name'],
            'type': move_data ['type']['name'],
            'power': move_data['power'],
            'accuracy': move_data['accuracy'],
            'power points': move_data['pp'],
            'effect': move_data ['effect_entries'][0]['short_effect'],
        }
        move_list.append(move_info)
    df_moves = pd.DataFrame(move_list)

else:
    print("nao funcionou")

Limpeza de dados

In [ ]:
df_moves.duplicated()
df_moves.drop_duplicates

In [ ]:
conn = sqlite3.connect('pokedex.db')
save_to_db(df_moves, 'Moves', conn)
conn.close()

Criação de Banco de Dados e conexão com SQL

In [ ]:
conn = sqlite3.connect('pokedex.db')
query = "SELECT * FROM Moves"
df = pd.read_sql(query, conn)
print(df)

conn.close()

Verificação dos bancos de dados criados para consumo no SQL

In [ ]:
conn = sqlite3.connect('pokedex.db')
query = "SELECT name FROM sqlite_master WHERE type='table'"
schema = pd.read_sql_query(query, conn)
print(schema)

conn.close()